In [6]:
import pandas as pd

import yahoo_scrape as scrape
import yahoo_sql as ysql

In [3]:
sql = ysql.fundamentals()
cnxn, cursor = sql.connect()

In [9]:
query_fundamentals = """
WITH
	CTE_LatestDate AS
	(
		SELECT 
			MAX(Date) AS [LatestDate],
			Category,
			Ticker
		FROM 
			[christopherFuru].[Yahoo].Fundamentals
		WHERE
			Category IN
			(
				'Enterprise Value/EBITDA',
				'Forward P/E'
			)
		GROUP BY
			Category,
			Ticker
	),

	CTE_LatestValues AS
	(
		SELECT
			LD.LatestDate,
			LD.Ticker,
			V.Category,
			V.Value
		FROM
			CTE_LatestDate LD
			LEFT JOIN christopherFuru.Yahoo.Fundamentals V ON
				V.[Date] = LD.LatestDate AND
				V.Ticker = LD.Ticker AND
				V.Category = LD.Category
	)

SELECT * FROM CTE_LatestValues
"""
query_company_info = """
SELECT
    *
FROM
    [christopherFuru].[Yahoo].[Companies]
"""

In [12]:
df_fundamentals = pd.read_sql(query_fundamentals, cnxn)
df_company_info = pd.read_sql(query_company_info, cnxn)

df_fundamentals = pd.merge(df_fundamentals, df_company_info, on = 'Ticker')
df_fundamentals

,LatestDate,Ticker,Category,Value,ISIN,CompanyName,Sector,List
0,2020-07-09,ERIC-A.ST,Forward P/E,22.17,SE0000108649,Ericsson A,Technology,Large cap
1,2020-07-09,ERIC-A.ST,Enterprise Value/EBITDA,14.89,SE0000108649,Ericsson A,Technology,Large cap
2,2020-07-09,ERIC-B.ST,Enterprise Value/EBITDA,14.89,SE0000108656,Ericsson B,Technology,Large cap
3,2020-07-09,ERIC-B.ST,Forward P/E,20.58,SE0000108656,Ericsson B,Technology,Large cap
4,2020-07-11,ESSITY-A.ST,Enterprise Value/EBITDA,10.52,SE0009922156,Essity A,Consumer Goods,Large cap
...,...,...,...,...,...,...,...,...
635,2020-07-09,EKTA-B.ST,Forward P/E,28.57,SE0000163628,Elekta B,Health Care,Large cap
636,2020-07-09,ELUX-A.ST,Enterprise Value/EBITDA,6.43,SE0000103806,Electrolux A,Consumer Goods,Large cap
637,2020-07-09,ELUX-A.ST,Forward P/E,24.45,SE0000103806,Electrolux A,Consumer Goods,Large cap
638,2020-07-09,ELUX-B.ST,Enterprise Value/EBITDA,6.43,SE0000103814,Electrolux B,Consumer Goods,Large cap
